In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline

from utils.visualize import *
from utils.FIR_filter import *
from utils.signal_process import *
from utils.preprocess import *
from utils.model import *
import utils.vision_transformer as VT

import torch
from torch.utils.data import Dataset, DataLoader

import pickle
import time

# Spectrogram

In [5]:
def prepare_data(dir, fs=10, features=['Q', 'omega', 'omega_l2', 'ANC'], start_pt=0, end_pt=-1, still_pt=300, after_still_pt=0, pool=1.0, d=0.05, window_size=128, stride=64, nperseg=128, noverlap=64, out_1=False, byCol=False):
    total_t = 0
    sensor_names=['imu1','imu2']
    cols = ['q_x', 'q_y', 'q_z', 'q_w']
    omega_axes = ['omega_u', 'omega_v', 'omega_w']
    ang_speed_cols = ['omega']
    anc_methods = ['LMS', 'LMS+LS', 'RLS', 'LRLS']

    q_col_ls = []
    omega_col_ls = []
    ang_speed_col_ls = []
    anc_col_ls = []

    for imu in sensor_names:
        for col in cols:
            q_col_ls.append(imu + "_" + col)
        for col in omega_axes:
            omega_col_ls.append(imu + "_" + col)
        for col in ang_speed_cols:
            ang_speed_col_ls.append(imu + "_" + col)

    for anc_method in anc_methods:
        for col in cols:
            anc_col_ls.append(anc_method + "_" + col)

    spectrograms, gts = [], []
    # iterate all files
    for file in os.listdir(dir):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"): 
            print(os.path.join(dir, filename))
            
            # load data
            data = pd.read_csv(os.path.join(dir, filename))
            data.columns = [
                "Timestamp",
                "imu1_q_x",
                "imu1_q_y",
                "imu1_q_z",
                "imu1_q_w",
                "imu2_q_x",
                "imu2_q_y",
                "imu2_q_z",
                "imu2_q_w",
                "Force",
                "RR",
            ]

            data = data.iloc[start_pt:end_pt]

            # align delay
            data = sp.align_delay(data, delay=10)
            data["Timestamp"] = pd.to_datetime(data["Timestamp"])
            data = data.set_index("Timestamp")

            # align IMU
            q_corr = sp.Q_RANSAC(data[0:still_pt], pool=pool, d=d)
            target, skew = 'imu1', 'imu2'
            Q_skew = data[[skew + '_q_x', skew + '_q_y', skew + '_q_z', skew + '_q_w']].to_numpy()
            Q_aligned = sp.align_quaternion(q_corr, Q_skew) # (sample num, 4)
            data_aligned = data.copy()

            for i, col in enumerate(cols):
                data_aligned[[skew + '_' + col]] = Q_aligned[:, i].reshape(-1, 1)

            # specify data range
            data_sml = data_aligned.copy() # data used in sml
            data_anc = data_aligned.copy() # data used in anc
            data_sml = data_sml[still_pt+after_still_pt:]

            start_time = time.perf_counter()

            # calculate omega features
            if 'omega' in features:
                for imu in sensor_names:
                    q = data_sml[[imu + "_" + "q_x", imu + "_" + "q_y", imu + "_" + "q_z", imu + "_" + "q_w"]].values # (sample num, 4)
                    omega = sp.q_to_omega(q)
                    ang_speed = sp.omega_to_AngSpeed(omega)
                    for i, omega_axis in enumerate(omega_axes):
                        data_sml.loc[:, imu + "_" + omega_axis] = omega[:, i]
                    
                    data_sml.loc[:, imu + "_" + ang_speed_cols[0]] = ang_speed

            # calculate ANC features
            if 'ANC' in features:
                anc_outputs = sp.anc_process(data_anc, NTAPS=3, LEARNING_RATE=0.001, delta=1, lam_rls=0.9995, epsilon=1e-6, lam_lrls=0.9995)
                for anc_output in anc_outputs:
                    method = anc_output['method']
                    for col in cols:
                        data_sml.loc[:, method + "_" + col] = anc_output[col][still_pt+after_still_pt:]

                
            # print(f'anc_outputs:{anc_outputs}')            

            # Q: (sample_num, channel_num)
            features_cols = []
            if 'Q' in features:
                features_cols.extend(q_col_ls)
            if 'omega' in features:
                features_cols.extend(omega_col_ls)
            if 'omega_l2' in features:
                features_cols.extend(ang_speed_col_ls)
            if 'ANC' in features:
                features_cols.extend(anc_col_ls)
            Q = data_sml[features_cols].values
            # print(f"features_cols:{features_cols}")
            # print(f'Q.shape:{Q.shape}')
            segmented_spectrograms, segmented_gt = sp.segment_data(Q, data_sml["Force"], window_size=window_size, stride=stride, nperseg=nperseg, noverlap=noverlap, out_1=out_1)

            # segmented_spectrograms: (num_windows, num_spectrograms, freq_bins, time_steps)
            # min-Max normalization
            index_sp = 0
            if 'Q' in features:
                segmented_spectrograms[:, index_sp:index_sp+8] = sp.normalize_spectrogram(segmented_spectrograms[:, index_sp:index_sp+8], byCol=byCol)
                index_sp += 8
            if 'omega' in features:
                segmented_spectrograms[:, index_sp:index_sp+6] = sp.normalize_spectrogram(segmented_spectrograms[:, index_sp:index_sp+6], byCol=byCol)
                index_sp += 6
            if 'omega_l2' in features:
                segmented_spectrograms[:, index_sp:index_sp+2] = sp.normalize_spectrogram(segmented_spectrograms[:, index_sp:index_sp+2], byCol=byCol)
                index_sp += 2
            if 'ANC' in features:
                segmented_spectrograms[:, index_sp:index_sp+4] = sp.normalize_spectrogram(segmented_spectrograms[:, index_sp:index_sp+4], byCol=byCol)
                segmented_spectrograms[:, index_sp+4:index_sp+8] = sp.normalize_spectrogram(segmented_spectrograms[:, index_sp+4:index_sp+8], byCol=byCol)
                segmented_spectrograms[:, index_sp+8:index_sp+12] = sp.normalize_spectrogram(segmented_spectrograms[:, index_sp+8:index_sp+12], byCol=byCol)
                segmented_spectrograms[:, index_sp+12:index_sp+16] = sp.normalize_spectrogram(segmented_spectrograms[:, index_sp+12:index_sp+16], byCol=byCol)
                index_sp += 16
            
            # print(f'sepctrograms:{segmented_spectrograms.shape}')
            # print(f'gt:{segmented_gt.shape}')
            # exit()
            spectrograms.append(segmented_spectrograms)
            gts.append(segmented_gt)

            end_time = time.perf_counter()
            total_t += end_time - start_time
    
    spectrograms = np.concatenate(spectrograms, axis=0)
    gts = np.concatenate(gts, axis=0)

    print('----------------------------')
    print(f'sepctrograms:{spectrograms.shape}')
    print(f'gt:{gts.shape}')

    return total_t

In [6]:
path_test = './data/17P/test/'

# 2-D spectrogram
window_size=256
stride=64
nperseg=128
noverlap=64
out1=True
byCol=True
features = ['Q', 'omega', 'omega_l2', 'ANC']

spectrogram_t = prepare_data(path_test, features=features, window_size=window_size, stride=stride, nperseg=nperseg, noverlap=noverlap, out_1=out1, byCol=byCol)

./data/17P/test/run_0520_0735.csv
best_score/total: 110/300
./data/17P/test/run_0520_1036.csv
best_score/total: 79/300
./data/17P/test/run_0604_0714.csv
best_score/total: 80/300
./data/17P/test/run_0604_0733.csv
best_score/total: 220/300
./data/17P/test/run_0621_0427.csv
best_score/total: 50/300
./data/17P/test/run_0629_0743.csv
best_score/total: 216/300
./data/17P/test/run_0629_0749.csv
best_score/total: 67/300
./data/17P/test/sit_0520_0719.csv
best_score/total: 176/300
./data/17P/test/sit_0520_1022.csv
best_score/total: 300/300
./data/17P/test/sit_0604_0658.csv
best_score/total: 237/300
./data/17P/test/sit_0621_0412.csv
best_score/total: 178/300
./data/17P/test/sit_0629_0730.csv
best_score/total: 74/300
./data/17P/test/stand_0520_0724.csv
best_score/total: 300/300
./data/17P/test/stand_0520_1027.csv
best_score/total: 300/300
./data/17P/test/stand_0604_0703.csv
best_score/total: 288/300
./data/17P/test/stand_0621_0417.csv
best_score/total: 300/300
./data/17P/test/stand_0629_0734.csv
b

In [7]:
spectrogram_t

9.358753200000137

# ML methods

In [9]:
# Load dataset
dataset_dir = "dataset/"
dataset_name = "17P_32"
pkl_test = pickle.load(open(os.path.join(dataset_dir, f'{dataset_name}_test.pkl'), 'rb'))
input_test, gt_test = pkl_test['input'], pkl_test['gt']

num_channels = input_test.shape[1]
num_freq_bins = input_test.shape[2]
num_time_steps = input_test.shape[3]

dataset_test = IMUSpectrogramDataset(input_test, gt_test)
test_loader = DataLoader(dataset_test, batch_size=1, shuffle=True)

MAE_32 = []

# Load models
# 2-D spectrogram
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
models_name = ['MLP_17P_32', 'CNN_17P_32', 'VT_17P_32_s']
models_name_show = ["MLP", "CNN", "ViT"]

models = [MLP_out1(num_freq_bins, num_time_steps, num_channels=num_channels),
          CNN_out1_2(num_channels=num_channels),
          VT.ViTRegression(in_channels=num_channels, patch_size=(3, 3), emb_dim=64, mlp_dim=128, num_heads=2, num_layers=2, device=device, load=True, model_name='VT_17P_32_s'),]

for i in range(len(models_name)):
    start_time = time.perf_counter()
    models[i].load_state_dict(torch.load(f'./models/{str(models_name[i])}.pt'))
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Evaluate model in whole testing set
    mse, mae = evaluate_model(models[i], test_loader, model_name=models_name[i], device=device)
    MAE_32.append(mae)

    end_time = time.perf_counter()
    delta_t = end_time - start_time
    total_params = sum(p.numel() for p in models[i].parameters() if p.requires_grad)

    print(f"{models_name[i]} Execution time: {(spectrogram_t+ delta_t):.4f} seconds")
    print(f"{models_name[i]} Total trainable parameters: {total_params}")

MLP_17P_32 Evaluation Results - MSE Loss: 20.6335, L1 Loss: 3.0929 1/min
MLP_17P_32 Execution time: 9.6396 seconds
MLP_17P_32 Total trainable parameters: 3326977
CNN_17P_32 Evaluation Results - MSE Loss: 17.2041, L1 Loss: 3.0686 1/min
CNN_17P_32 Execution time: 9.7578 seconds
CNN_17P_32 Total trainable parameters: 388673
VT_17P_32_s Evaluation Results - MSE Loss: 18.5271, L1 Loss: 3.0919 1/min
VT_17P_32_s Execution time: 10.0681 seconds
VT_17P_32_s Total trainable parameters: 85697
